In [ ]:
# ollama won't work because it doesn't support tool calls yet
from typing import Annotated, Sequence, TypedDict
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage # The foundational class for all message types in LangGraph
from langchain_core.messages import ToolMessage # Passes data back to LLM after it calls a tool such as the content and the tool_call_id
from langchain_core.messages import SystemMessage # Message for providing instructions to the LLM
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


load_dotenv()

# Annotated - provides additional context to your variable or key without affecting the type itself
# here is an email is a type of string and "" provides some more additional information, additional context
# And this is basically adding onto the metadata of this key or variable
# email = Annotated[str, "This has to be a valid email format"]


# Sequence - To automatically handle the state updates for sequences such as by adding new message to chat history


# add_messages is a reducer function,
# Reducer Function is: Rule that control how updates from nodes are combined with existing state.
# Tells us how to merge new data into current state
# without a reducer, updates would have replaced the exciting value (state) entirely
"""
# Without a reducer
state = {"messages": ["Hi!"]}
update = {"messages":["Nice to meet you!"]}
new_state = {"messages":["Nice to meet you!"]}
# With a reducer
state = {"messages": ["Hi!"]}
update = {"messages": ["Nice to meet you!"]}
new_state = {"messages": ["Hi!", "Nice to meet you!"]}
"""




class AgentState(TypedDict):
    # preserve the state
    messages: Annotated[Sequence[BaseMessage], add_messages]


# tool: this is a decorator and decorator basically tells Python that this function is a quite special.
# It does something and well it is special because it's a tool which we're going to use
@tool
def add(a: int, b: int):
    # this docstring is very important here!!!!!!!!!!!!!!
    """This is an addition function that add 2 numbers together"""

    return a + b


@tool
def subtract(a: int, b: int):
    """Subtraction function"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiplication function"""
    return a * b



tools = [add, subtract, multiply]

# bind_tools --> to tell chat-gpt a tool that it can be used by it
model = ChatOpenAI(model='gpt-4o').bind_tools(tools)


def model_call(state: AgentState)-> AgentState:
    system_prompt = SystemMessage(content="You are my AI assistant, please answer my query to the best of your ability.")
    # state['messages'] --> my query (human message)
    response = model.invoke([system_prompt] + state['messages'])
    # another way for updating state and return it
    return {"messages" : [response]}


def should_continue(state: AgentState):
    messages = state['messages']
    last_message = messages[-1]
    if not last_message.tool_calls:
        return "end"
    else:
        return "continue"


graph = StateGraph(AgentState)

graph.add_node("our_agent", model_call)

# The tool node essentially is just a singular node which contains all the different tools.
tool_node = ToolNode(tools= tools)
graph.add_node("tools", tool_node)

graph.set_entry_point("our_agent")

graph.add_conditional_edges(
    "our_agent", # source node
    should_continue,
    {
        # "Edge" : "Node"
        "continue" : "tools",
        "end" : END
    }

)

graph.add_edge("tools", "our_agent")

app = graph.compile()


def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()


inputs = {"messages": [("user", "Add 40 + 12 and then multiply then result by 6 and then subtract it by 1. Also tell me a joke please.")]}
print_stream(app.stream(inputs, stream_mode="values"))

C:\Users\abdel\AppData\Local\Temp\ipykernel_14016\1639978558.py:75: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import OllamaLLM``.
  model = Ollama(


AttributeError: 'Ollama' object has no attribute 'bind_tools'